In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import datetime as dt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [3]:
finalSeries = []

for i in range(2000,2017):
    filename = "data/data"+str(i)
    for j in range(1,13):
        filename = filename + "month" + str(j) +".csv"
        data = pd.read_csv(filename, usecols=['NO2 Mean'])
        series = data['NO2 Mean'].tolist()
        if(len(series)<31 and len(series)!=0):
            while len(series)!=31:
                last = series[len(series)-1]
                series.append(last)
        if(len(series)!=0):
            series.append(j)
            seriesNP = np.asarray(series)
            finalSeries.append(seriesNP)
        filename = "data/data"+str(i)

finalSeriesNP = np.asarray(finalSeries)

In [4]:
def DTWDistance(s1, s2,w):
    DTW={}
    
    w = max(w, abs(len(s1)-len(s2)))
    
    for i in range(-1,len(s1)):
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0
  
    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
		
    return np.sqrt(DTW[len(s1)-1, len(s2)-1])

In [5]:
def LB_Keogh(s1,s2,r):
    LB_sum=0
    for ind,i in enumerate(s1):
        
        lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        
        if i>upper_bound:
            LB_sum=LB_sum+(i-upper_bound)**2
        elif i<lower_bound:
            LB_sum=LB_sum+(i-lower_bound)**2
    
    return np.sqrt(LB_sum)

In [16]:
from sklearn.metrics import classification_report

def knn(train,test,w):
    preds=[]
    for ind,i in enumerate(test):
        min_dist=float('inf')
        closest_seq=[]
        #print ind
        for j in train:
            if LB_Keogh(i[:-1],j[:-1],5)<min_dist:
                dist=DTWDistance(i[:-1],j[:-1],w)
                if dist<min_dist:
                    min_dist=dist
                    closest_seq=j
        preds.append(closest_seq[-1])
    return classification_report(test[:,-1],preds), preds

In [14]:
#np.random.shuffle(finalSeriesNP)
train, test = finalSeriesNP[:150,:], finalSeriesNP[150:,:]


In [18]:
report, pred = knn(train,test,10)

In [31]:
print(pred)
print(test[:,-1])
print(report)

[8.0, 3.0, 10.0, 1.0, 4.0, 5.0, 9.0, 2.0, 3.0, 1.0, 3.0, 8.0, 5.0, 10.0, 9.0, 3.0, 2.0, 12.0, 1.0, 12.0, 10.0, 7.0, 6.0, 6.0, 3.0, 7.0, 8.0, 10.0, 8.0, 11.0, 6.0, 5.0, 3.0, 3.0, 7.0, 5.0, 6.0, 7.0, 11.0, 3.0, 3.0, 1.0, 6.0]
[  6.   8.  10.   1.   9.   5.   8.  10.  12.   2.  11.   9.   4.  12.   4.
   5.   5.  12.   2.   2.   5.   5.   7.   8.   2.   9.   7.  12.   6.  10.
   7.   7.  10.   6.   7.   7.   3.   6.   1.   3.  10.  11.   9.]
             precision    recall  f1-score   support

        1.0       0.25      0.50      0.33         2
        2.0       0.00      0.00      0.00         4
        3.0       0.11      0.50      0.18         2
        4.0       0.00      0.00      0.00         2
        5.0       0.25      0.20      0.22         5
        6.0       0.00      0.00      0.00         4
        7.0       0.25      0.17      0.20         6
        8.0       0.00      0.00      0.00         3
        9.0       0.00      0.00      0.00         4
       10.0       0.25    